In [1]:
System.getProperty("java.version")

2023-04-26 17:17:15.894 [IJava-executor-0] DEBUG test-jshell-log - --------------- test-jshell-log debug ------------------
2023-04-26 17:17:15.898 [IJava-executor-0] ERROR test-jshell-log - --------------- test-jshell-log error ------------------
2023-04-26 17:17:15.901 [IJava-executor-0] INFO  test-jshell-log - log to file: /var/log/jshell/jshell-20230426_171715.390.log


System.getProperty("java.version...: 20.0.1

In [2]:
// System.getProperty("jdk.virtualThreadScheduler.parallelism")

In [8]:
// 限制虚拟线程的平台线程为一个，方便下面查看 synchronized 锁的缺点
System.setProperty("jdk.virtualThreadScheduler.parallelism", "1");
System.setProperty("jdk.virtualThreadScheduler.maxPoolSize", "1");

System.setProperty("jdk.virtualT...: 1

## 1. base intro

compare:
- Platform thread:
    - 创建和启动时间 ms 级别；
    - jdk8 下需要 1MB 以上的 stack 内存;
    - 与 OS 线程是一对一；
    - 由 os 调度；
- Virtual thread:
    - 创建和启动时间 ns 级别；
    - stack 内存动态伸缩；
    - 使用一个特殊的 Platform thread 线程池（ForkJoinPool 的 FIFO 模式）来调度，称为 carrier threads；
    - 由 jvm 调度，jdk 重写了阻塞，阻塞时最终会调用 `VirtualThread.park`，从 carrier threads 上卸载(`synchronized` 的阻塞是汇编实现，所以 VT 无法调度，最终会直接阻塞 carrier threads)

[Virtual thread 的调度线程相关参数](https://docs.oracle.com/en/java/javase/19/docs/api/java.base/java/lang/Thread.html#inheritance):
- `jdk.virtualThreadScheduler.parallelism`: Platform thread 数量，默认为处理器核数；
- `jdk.virtualThreadScheduler.maxPoolSize`: 最大 Platform thread 数量，默认为 256

使用时注意:
- 虚拟线程 仅适合 io密集型，与 Fork Join 互补;
- 虚拟线程 速度不会比 Platform thread 快，因为其调度在 Platform thread 之上；
- 虚拟线程 中加锁尽量不使用 `synchronized`，因为其会阻塞底层的 Platform thread；
- 虚拟线程 不可复用，所以不要使用线程池，为兼容旧的池化写法，使用 `Thread.ofVirtual().factory()` 或者直接使用 `Executors.newVirtualThreadPerTaskExecutor()`，其每个任务会新创建虚拟线程；
- 虚拟线程 下使用 ThreadLocal 要注意，由于虚拟线程不复用数量大，每次创建 ThreadLocal 对象会非常耗费资源

### 1.1 quick start

In [4]:
// platform thread
var pthread = Thread.ofPlatform()
        .name("platform-", 0)
        .start(() -> {
            System.out.println("platform " + Thread.currentThread());
        });
pthread.join();

// virtual thread
var vthread = Thread.ofVirtual()
        .name("virtual-", 0)
        .start(() -> {
            System.out.println("virtual " + Thread.currentThread());
        });
vthread.join();

platform Thread[#28,platform-0,5,main]
virtual VirtualThread[#29,virtual-0]/runnable@ForkJoinPool-1-worker-1


In [9]:
import java.util.stream.IntStream;


var vtList = IntStream.range(0, 20).boxed()
    .map(idx -> Thread.ofVirtual().name(String.format("vt-%3d", idx)).unstarted(() -> System.out.println(Thread.currentThread())))
    .toList();

vtList.forEach(Thread::start);
Thread.sleep(5000);

VirtualThread[#232,vt-  0]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#233,vt-  1]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#234,vt-  2]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#235,vt-  3]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#236,vt-  4]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#237,vt-  5]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#238,vt-  6]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#239,vt-  7]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#240,vt-  8]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#241,vt-  9]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#242,vt- 10]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#243,vt- 11]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#244,vt- 12]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#245,vt- 13]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#246,vt- 14]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#247,vt- 15]/runnable@ForkJoinPool-1-worker-2
VirtualThread[#248,vt- 16]/runnable@Fork

### 1.2 lock

In [12]:
import java.time.Duration;
import java.time.LocalTime;
import java.time.Instant;
import java.time.temporal.ChronoUnit;
import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;


void sleep(Duration duration) {
    try {
        Thread.sleep(duration.toMillis());
    } catch (Exception e) {
        throw new RuntimeException(e);
    }
}

void lockAndCall(Lock lock, Runnable runable) {
    lock.lock();
    try {
        runable.run();
    } finally {
        lock.unlock();
    }
}

#### a. ReentrantLock only lock current thread

In [13]:
Lock lock = new ReentrantLock();
int counter = 0;

var vtList1 = List.of(
    Thread.ofVirtual()
        .name("vt_nolock-", 0)
        .unstarted(() -> {
            while (counter < 5) {
                System.out.printf("sec=%d, virtual thread=%s%n", System.currentTimeMillis() / 1000, Thread.currentThread().getName());
                sleep(Duration.ofSeconds(1));
            }
        }),

    Thread.ofVirtual()
        .name("vt-", 0)
        .unstarted(() -> {
            while (counter < 5) {
                lockAndCall(lock, () -> {
                    System.out.printf("sec=%d, virtual thread=%s%n", System.currentTimeMillis() / 1000, Thread.currentThread().getName());
                    counter++;
                });
                sleep(Duration.ofSeconds(1));
            }
        }),

    Thread.ofVirtual()
        .name("vt-", 1)
        .unstarted(() -> {
            lockAndCall(lock, () -> sleep(Duration.ofSeconds(3)));
            System.out.printf("sec=%d, virtual thread=%s%n", System.currentTimeMillis() / 1000, Thread.currentThread().getName());
        })
);

Instant begin = Instant.now();
vtList1.forEach(Thread::start);
for (var vt : vtList1) vt.join();
Instant end = Instant.now();
System.out.println("Duration = " + Duration.between(begin, end));

sec=1682500883, virtual thread=vt_nolock-0
sec=1682500883, virtual thread=vt-0
sec=1682500884, virtual thread=vt_nolock-0
sec=1682500885, virtual thread=vt_nolock-0
sec=1682500886, virtual thread=vt-1
sec=1682500886, virtual thread=vt-0
sec=1682500886, virtual thread=vt_nolock-0
sec=1682500887, virtual thread=vt-0
sec=1682500887, virtual thread=vt_nolock-0
sec=1682500888, virtual thread=vt-0
sec=1682500888, virtual thread=vt_nolock-0
sec=1682500889, virtual thread=vt-0
Duration = PT7.757528651S


#### b. synchronized will lock carrier thread

In [14]:
Object lock = new Object();
int counter = 0;

var vtList1 = List.of(
    Thread.ofVirtual()
        .name("vt_nolock-", 0)
        .unstarted(() -> {
            while (counter < 5) {
                System.out.printf("sec=%d, virtual thread=%s%n", System.currentTimeMillis() / 1000, Thread.currentThread().getName());
                sleep(Duration.ofSeconds(1));
            }
        }),

    Thread.ofVirtual()
        .name("vt-", 0)
        .unstarted(() -> {
            while (counter < 5) {
                synchronized (lock) {
                    System.out.printf("sec=%d, virtual thread=%s%n", System.currentTimeMillis() / 1000, Thread.currentThread().getName());
                    counter++;
                }
                sleep(Duration.ofSeconds(1));
            }
        }),

    Thread.ofVirtual()
        .name("vt-", 1)
        .unstarted(() -> {
            synchronized (lock) {
                sleep(Duration.ofSeconds(3));
            }
            System.out.printf("sec=%d, virtual thread=%s%n", System.currentTimeMillis() / 1000, Thread.currentThread().getName());
        })
);

Instant begin = Instant.now();
vtList1.forEach(Thread::start);
for (var vt : vtList1) vt.join();
Instant end = Instant.now();
System.out.println("Duration = " + Duration.between(begin, end));

sec=1682500891, virtual thread=vt_nolock-0
sec=1682500891, virtual thread=vt-0
sec=1682500894, virtual thread=vt-1
sec=1682500894, virtual thread=vt_nolock-0
sec=1682500894, virtual thread=vt-0
sec=1682500895, virtual thread=vt_nolock-0
sec=1682500895, virtual thread=vt-0
sec=1682500896, virtual thread=vt_nolock-0
sec=1682500896, virtual thread=vt-0
sec=1682500897, virtual thread=vt_nolock-0
sec=1682500897, virtual thread=vt-0
Duration = PT7.213763751S


### 3. Executors

In [31]:
ExecutorService executor = Executors.newVirtualThreadPerTaskExecutor();
executor.execute(() -> System.out.println(Thread.currentThread()));
sleep(Duration.ofMillis(100));

VirtualThread[#278]/runnable@ForkJoinPool-1-worker-4


VirtualThread[#277]/runnable@ForkJoinPool-1-worker-4
